In [231]:
import sys
sys.path.append('..')
import sql
import pandas as pd
from functools import reduce
from tabulate import tabulate
import datetime
import matplotlib.pyplot as plt
import numpy as np

In [232]:
csv = pd.read_csv('final-frozen.csv')

In [233]:
d= {'Is there a documentation source?': 'is_doc','is the fix mentioned?':'is_fix', 'is there unrelated change mentioned?': 'is_uc', 'is breaking change mentioned?':'is_br' , 'main source' :'doc_source', 'How is the fix mentioned? (final codes); separated by semicolon':'fix_pattern',
'how the breaking change is mentioned? (final codes); separated by semicolon':'breaking_pattern', 'main source':'source'}
aniqa = pd.read_csv('process-frozen-aniqa.csv')
aniqa = aniqa.rename(columns = d)
aniqa['is_uc']= aniqa['is_uc'].str.lower()
nasif = pd.read_csv('process-frozen-nasif.csv')
nasif = nasif.rename(columns = d)
nasif['is_uc']=nasif['is_uc'].str.lower()

In [234]:
resolved = csv[csv['agreed']=='resolved']
resolved = resolved.to_dict('records')

In [235]:
c =a = 0
for row in resolved:
    r = row['row']
    if nasif.iloc[r]['is_doc'] != aniqa.iloc[r]['is_doc']:
        if row['is_doc']=='y':
            a+= 1
        c+=1
c,a

(28, 26)

In [236]:
csv = csv[csv['is_doc']=='y'].to_dict('records')
len(csv)

363

In [237]:
def cohenkappayesno(yy,yn,ny,nn):
    agreed = yy + nn
    total = agreed + yn + ny
    ef1 = ((yy+yn)*(yy+ny))/total
    ef2 = ((ny+nn)*(yn+nn))/total
    ef = ef1 + ef2
    return (agreed - ef)/(total - ef)



In [238]:
yy = yn = ny = nn = 0
for row in csv:
    r = row['row'] - 2
    if nasif.iloc[r]['is_uc'] == 'y' and aniqa.iloc[r]['is_uc'] == 'y':
        yy += 1
    elif nasif.iloc[r]['is_uc'] == 'y' and aniqa.iloc[r]['is_uc'] == 'n': 
        yn+=1
    elif nasif.iloc[r]['is_uc'] == 'n' and aniqa.iloc[r]['is_uc'] == 'y':
        ny+=1
    elif nasif.iloc[r]['is_uc']== 'n' and aniqa.iloc[r]['is_uc'] == 'n':
        nn+=1
    else:
        nn+1
cohenkappayesno(yy,yn,ny,nn)

0.819233550253073

In [239]:
codes = ['vulnerability description', 'fix reference', 'security notice',
       'affected version listed', 'advisory reference', 'exploit',
       'affected component listed', 'affected configuration']
d= []
for c in codes:
    yy = yn = ny = nn = 0
    for row in csv:
        r = row['row'] - 2
        n = nasif.iloc[r]['fix_pattern']
        n = str(n).split(';')
        for i,e in enumerate(n):
            n[i] = e.lower().strip()
        a = aniqa.iloc[r]['fix_pattern']
        a = str(a).split(';')
        for i,e in enumerate(a):
            a[i] = e.lower().strip()
        if c in n and c in a:
            yy+=1
        if c in n and c not in a:
            yn+=1
        if c not in n and c in a:
            ny+=1
        else:
            nn+=1
    print(c, cohenkappayesno(yy,yn,ny,nn))
    d.append({'code':c, 'kappa':round(cohenkappayesno(yy,yn,ny,nn),2) })
    

vulnerability description 0.6691343484478843
fix reference 0.5284013605442177
security notice 0.5996669651413042
affected version listed 0.41259156602652974
advisory reference 0.8077440679966946
exploit 0.319464516619761
affected component listed 0.3887680174054935
affected configuration 0.4979423868312715


In [240]:
df = pd.DataFrame(d)
print(tabulate(df, tablefmt='latex', headers='keys',showindex=False))

\begin{tabular}{lr}
\hline
 code                      &   kappa \\
\hline
 vulnerability description &    0.67 \\
 fix reference             &    0.53 \\
 security notice           &    0.6  \\
 affected version listed   &    0.41 \\
 advisory reference        &    0.81 \\
 exploit                   &    0.32 \\
 affected component listed &    0.39 \\
 affected configuration    &    0.5  \\
\hline
\end{tabular}


In [241]:
codes = ['breaking change notice', 'code change reference',
       'affected api listed', 'action required', 'affected configuration']
for c in codes:
    yy = yn = ny = nn = 0
    for row in csv:
        r = row['row'] - 2
        n = nasif.iloc[r]['breaking_pattern']
        n = str(n).split(';')
        for i,e in enumerate(n):
            n[i] = e.lower().strip()
        a = aniqa.iloc[r]['breaking_pattern']
        a = str(a).split(';')
        for i,e in enumerate(a):
            a[i] = e.lower().strip()
        if c in n and c in a:
            yy+=1
        if c in n and c not in a:
            yn+=1
        if c not in n and c in a:
            ny+=1
        else:
            nn+=1
    print(c, cohenkappayesno(yy,yn,ny,nn), yy,yn,ny,nn)
    d.append({'code':c, 'kappa':round(cohenkappayesno(yy,yn,ny,nn),2) })

breaking change notice 0.833981841763943 16 5 1 362
code change reference 1.0 9 0 0 363
affected api listed 0.918953234111589 18 2 1 362
action required 0.8722966014418123 7 2 0 363
affected configuration 0.7231807951988005 4 2 1 362


In [242]:
df = pd.DataFrame(d)
print(tabulate(df, tablefmt='latex', headers='keys',showindex=False))


\begin{tabular}{lr}
\hline
 code                      &   kappa \\
\hline
 vulnerability description &    0.67 \\
 fix reference             &    0.53 \\
 security notice           &    0.6  \\
 affected version listed   &    0.41 \\
 advisory reference        &    0.81 \\
 exploit                   &    0.32 \\
 affected component listed &    0.39 \\
 affected configuration    &    0.5  \\
 breaking change notice    &    0.83 \\
 code change reference     &    1    \\
 affected api listed       &    0.92 \\
 action required           &    0.87 \\
 affected configuration    &    0.72 \\
\hline
\end{tabular}
